<a href="https://colab.research.google.com/github/raiaiaia/llm-code-review-clj/blob/main/default_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Model Experiment Default


#Instalações e importações necessárias

In [ ]:
%%capture
!pip install -q sentence_transformers \
                accelerate \
                transformers \
                bitsandbytes

In [1]:
!huggingface-cli login

In [2]:
!huggingface-cli whoami

In [ ]:
%%capture
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from transformers import BertTokenizer, BertModel
from transformers import AutoTokenizer, pipeline
from huggingface_hub import InferenceClient
from time import time
import transformers
import pandas as pd
import numpy as np
import argparse
import torch
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Carregando os dados

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/rayanne-llm-code-review-clj/Dataset/com-filtro/filtered_by_number_of_lines.csv')
df.head()

,pull_request_url,comment_id,diff_hunk,content,cleaned_diff_hunk,diff_lines
0,https://api.github.com/repos/adamtornhill/code...,33513420,"@@ -1,3 +1,5 @@+[![Build Status](https://travi...",Ah sorry there is clearly an issue here @adamt...,[![Build Status](https:,1
1,https://api.github.com/repos/adamtornhill/code...,95127516,"@@ -10,52 +10,106 @@ ""/some/path => G1 /anothe...",The test cases are a bit too noisy. We need to...,"(def ^:const multi-regexp-group-spec\n""^/some/...",9
2,https://api.github.com/repos/anmonteiro/lumo/p...,87304994,"@@ -35,7 +35,7 @@ Check out `lumo -h` for usag...",:+1:,"2. At the root of the repository, run: `boot r...",1
3,https://api.github.com/repos/anmonteiro/lumo/p...,88037369,"@@ -0,0 +1,35 @@+import net from 'net';",this needs a `/* @flow */` header,import net from 'net';,1
4,https://api.github.com/repos/anmonteiro/lumo/p...,88787684,"@@ -75,30 +76,35 @@ function consumeBuffer(buf...",can we keep the readline interface as the firs...,"export function processLine(sessionId: number,...",1


In [ ]:
df = df[df['diff_lines'] >= 5].reset_index(drop=True)

In [ ]:
print(df.shape)

(5105, 6)


#Preparando o ambiente

In [ ]:
MISTRAL_INSTRUCT= 'mistralai/Mistral-7B-Instruct-v0.2'
MISTRAL= 'mistralai/Mistral-7B-v0.1'

In [ ]:
quantization = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
    )

model = AutoModelForCausalLM.from_pretrained(
    MISTRAL_INSTRUCT,
    device_map="auto",
    quantization_config=quantization
)

model.eval()

tokenizer = AutoTokenizer.from_pretrained(
    MISTRAL_INSTRUCT,
    padding_side="left",
    model_max_length=4096
    )

tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
def format_prompt(code_snippet):
    return f'''
      <s>[ISNT] You are an experienced programmer in Clojure.
      Review the following code snippet and provide feedback on: readability, efficiency, maintainability, and potencial bugs.[/ISNT]

      {code_snippet}

      </s>

      [INST] Format your review as text with itemized concrete instructions to the author of the code. Do not add this prompt to the output. [/ISNT]
      '''

#Rodando o modelo

In [3]:
t = time()

df['model_comment'] = [None for _ in range(df.shape[0])]

for index, row in df.iterrows():
  print(f'Running example {index+1}')

  prompt = format_prompt(row['cleaned_diff_hunk'])
  model_input = tokenizer(prompt, return_tensors='pt').to('cuda')

  generated_ids = model.generate(
                                **model_input,
                                 temperature=0.1,
                                 top_k=1,
                                 top_p=1.0,
                                 repetition_penalty=1.4,
                                 min_new_tokens=16,
                                 max_new_tokens=512,
                                 do_sample=True
                                )

  resp = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
  clean_text = resp[(len(prompt)-13):]

  df.at[index, 'model_comment'] = clean_text
  if ((index+1) % 5 == 0):
    print(f'Saving results (current shape: {index+1})')
    df.to_csv('/content/drive/MyDrive/rayanne-llm-code-review-clj/Dataset/output/default_model_output.csv', index=False)

print(f'Demanded time: {time() - t} seconds')